In [32]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time 


# Cargar variables de entorno (tu GOOGLE_API_KEY desde el archivo .env)
load_dotenv()

True

In [33]:
# Credenciales de la Base de Datos ANTIGUA (leídas de tu descripción)
DB_OLD_USER = "root"
DB_OLD_PASS = "" # Sin contraseña
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA (leídas de tu descripción)
DB_NEW_USER = "root"
DB_NEW_PASS = "" # Sin contraseña
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")

STATIC_CONTENT_STRUCTURE = {
    "en": {
        "special_tag": "Urgent Help",
        'description_heading'    : 'Description Heading',
        'description_text'       : 'Detailed Description',
        'summary_title'          : 'Summary Title',
        'summary_text'           : 'Summary Text',
        'help_mini_title'        : "Help People In need",
        'help_title'             : "Start Donating Today, Make The Difference",
        'help_btn_text'          : "DONATE TO THIS PROJECT >"
    },
    "es": {
        "special_tag": "Ayuda urgente",
        "description_heading": "Subtitulo por defecto",
        "description_text": "Texto de descripción por defecto",
        'summary_title': 'Titulo de resumen',
        "summary_text": "Texto de resumen",
        'help_mini_title': "Ayuda a personas en necesidad",
        'help_title': "Comienza a donar hoy, Haz la diferencia",
        'help_btn_text': "DONAR A ESTE PROYECTO >"
    }
}

STATIC_CONTENT_JSON = json.dumps(STATIC_CONTENT_STRUCTURE, ensure_ascii=False)

with open("promptproject2.txt", 'r', encoding='utf-8') as f:
    system_prompt = f.read()

In [34]:
# --- FUNCIONES AUXILIARES ---

def call_azure_openai(system_prompt, user_message, max_retries=5):
    """Envía una solicitud a Azure OpenAI y maneja errores 429 con reintentos."""
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    api_key = os.getenv("OPENAI_API_KEY")

    if not endpoint or not api_key:
        print("❌ Faltan credenciales de Azure OpenAI")
        return None

    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }

    payload = {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        "max_tokens": 4000,
        "temperature": 0.3,
        "top_p": 0.95,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5
    }

    for attempt in range(max_retries):
        try:
            response = requests.post(endpoint, headers=headers, json=payload, timeout=60)

            if response.status_code == 429:
                # Si Azure devuelve un retry-after personalizado, lo usamos
                retry_after = response.headers.get("Retry-After")
                wait_time = int(retry_after) if retry_after else 2 ** attempt
                print(f"🔁 Esperando {wait_time} segundos por límite de tasa (429)...")
                time.sleep(wait_time)
                continue

            if response.status_code != 200:
                print(f"❌ Error {response.status_code}: {response.text}")
                return None

            return response.json()["choices"][0]["message"]["content"]

        except requests.exceptions.RequestException as e:
            print(f"❌ Error en la solicitud: {e}")
            return None
        except (KeyError, IndexError, json.JSONDecodeError) as e:
            print(f"❌ Error procesando respuesta: {e}")
            return None

    print("❌ Se excedieron los reintentos por error 429.")
    return None

def clean_html(html_content):
    """Función principal de limpieza con truncamiento opcional"""
    simplified = simplify_html(html_content)
    
    # Truncar si es necesario después de simplificar
    MAX_LENGTH = 10000  # Menor que antes porque el HTML ahora es más simple
    if len(simplified) > MAX_LENGTH:
        # Truncar de manera inteligente manteniendo estructura
        soup = BeautifulSoup(simplified, 'html.parser')
        text = soup.get_text()[:MAX_LENGTH]
        return text + " [CONTENIDO TRUNCADO]"
    
    return simplified

def simplify_html(html_content):
    """
    Simplifica el HTML eliminando:
    - Estilos (style, class, id)
    - Scripts y metadatos
    - Comentarios y doctypes
    - Atributos innecesarios
    - Conserva solo estructura básica y contenido
    """
    if not html_content or pd.isna(html_content):
        return ""
    
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Remover elementos no deseados
        for element in soup(['script', 'style', 'meta', 'link', 'head', 'noscript', 'iframe']):
            element.decompose()
        
        # Remover comentarios y doctypes
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()
        
        for doctype in soup.find_all(string=lambda text: isinstance(text, Doctype)):
            doctype.extract()
        
        # Simplificar etiquetas conservando solo atributos esenciales
        for tag in soup.find_all(True):
            # Conservar solo estas etiquetas básicas
            allowed_tags = ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 
                            'ul', 'ol', 'li', 'a', 'strong', 'em', 'b', 'i',
                            'table', 'tr', 'td', 'th', 'div', 'span', 'br', 'hr']
            
            if tag.name not in allowed_tags:
                tag.unwrap()  # Conserva el contenido pero elimina la etiqueta
                continue
                
            # Mantener solo estos atributos (opcional)
            allowed_attrs = ['href']
            attrs = {}
            for attr in allowed_attrs:
                if attr in tag.attrs:
                    attrs[attr] = tag.attrs[attr]
            tag.attrs = attrs
            
            # Eliminar estilos en línea
            if 'style' in tag.attrs:
                del tag.attrs['style']
        
        # Simplificar estructura
        for tag in soup.find_all(['div', 'span']):
            tag.unwrap()
        
        # Reducir espacios innecesarios
        output = str(soup)
        output = re.sub(r'\n\s+', '\n', output)  # Eliminar espacios múltiples
        output = re.sub(r'\n{3,}', '\n\n', output)  # Reducir saltos de línea múltiples
        
        return output
    
    except Exception as e:
        print(f"⚠️ Error simplificando HTML: {e}")
        # Devolver versión truncada como fallback
        return html_content[:5000] + " [HTML SIMPLIFICADO]"

def create_db_engine(user, password, host, db_name):
    """Crea un motor de conexión SQLAlchemy."""
    # Codifica la contraseña si contiene caracteres especiales
    password_encoded = quote_plus(password)
    connection_string = f"mysql+mysqlconnector://{user}:{password_encoded}@{host}/{db_name}"
    return create_engine(connection_string)


def slugify(text):
    """Convierte un texto en un slug limpio, ideal para URLs."""
    text = re.sub(r'[^\w\s-]', '', text)  # Elimina caracteres especiales
    return re.sub(r'[-\s]+', '-', text.lower()).strip('-_')

def load_prompt_template(filepath="promptproject2.txt"):
    """Carga la plantilla del prompt desde un archivo."""
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.read()

# --- FASE 1: EXTRACCIÓN ---

# def extract_from_old_db():
#     """Extrae los datos de la tabla 'pages' de la BD antigua."""
#     print("Iniciando Fase 1: Extracción de datos...")
#     engine_old = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME)
    
#     # Columnas exactas que solicitaste
#     query = """
#     SELECT 
#         id, title, excerpt, body, meta_description, 
#         `IFRAME`, `IFRAMEES`, video, videoes, summary_es,
#         summary, problem, solution, longterm
#     FROM pages
#     WHERE layout = 'campaign';
#     """
    
#     try:
#         df = pd.read_sql(query, engine_old)
#         print(f"✅ Extracción completada. Se encontraron {len(df)} registros.")
#         return df
#     except Exception as e:
#         print(f"❌ Error al extraer datos: {e}")
#         return pd.DataFrame() # Devuelve un DataFrame vacío si falla

# --- FASE 1: EXTRACCIÓN (MODIFICADA) ---

def get_page_translations(page_id, engine):
    """Obtiene todas las traducciones para un ID de página y las pivota a un diccionario."""
    query = text("""
        SELECT column_name, value 
        FROM translations 
        WHERE table_name = 'pages' AND foreign_key = :page_id AND locale = 'es'
    """)
    
    # NUEVO: Diccionario para almacenar traducciones de la página actual
    translations = {}
    try:
        with engine.connect() as connection:
            # Ejecutamos la consulta por cada página
            result = connection.execute(query, {'page_id': page_id})
            for row in result:
                # Mapeamos 'nombre_columna' -> 'valor_traducido'
                translations[row.column_name] = row.value
    except Exception as e:
        print(f"⚠️  No se pudieron obtener traducciones para el ID {page_id}: {e}")
        
    return translations


def extract_from_old_db():
    """Extrae datos de 'pages' y AÑADE las traducciones existentes desde 'translations'."""
    print("Iniciando Fase 1: Extracción de datos y traducciones...")
    engine_old = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME)
    
    # La consulta a 'pages' sigue siendo la misma
    query = """
    SELECT 
        id, title, excerpt, body, meta_description, 
        `IFRAME`, `IFRAMEES`, video, videoes, summary_es,
        summary, problem, solution, longterm
    FROM pages
    WHERE layout = 'campaign';
    """
    
    try:
        df = pd.read_sql(query, engine_old)
        print(f"✅ Extracción de 'pages' completada. Se encontraron {len(df)} registros.")
        
        # NUEVO: Iterar para obtener traducciones de cada página
        translations_list = []
        for page_id in df['id']:
            translations_list.append(get_page_translations(page_id, engine_old))
            
        # Añadimos las traducciones como una nueva columna que contiene un diccionario
        df['translations_es'] = translations_list
        
        print("✅ Traducciones asociadas correctamente.")
        return df
        
    except Exception as e:
        print(f"❌ Error al extraer datos: {e}")
        return pd.DataFrame()

# --- FASE 2: TRANSFORMACIÓN ---

# def process_with_ai(html_content, prompt_template):
#     """Procesa el contenido HTML con Azure OpenAI para obtener un JSON."""
#     if not html_content or pd.isna(html_content):
#         return json.dumps({"en": "", "es": ""})
    
#     # Limpiar y simplificar HTML
#     cleaned_html = clean_html(html_content)
    
#     # DEBUG: Ver reducción de tamaño
#     orig_size = len(html_content)
#     new_size = len(cleaned_html)
#     print(f"  Reducción HTML: {orig_size} → {new_size} caracteres (-{100*(1-new_size/orig_size):.1f}%)")
    
#     # Dividir el prompt en sistema y contenido de usuario
#     system_prompt = prompt_template.split("---")[0].strip()
#     user_message = f"CONTENIDO HTML A PROCESAR:\n{cleaned_html}"
    
#     # Llamar a Azure OpenAI
#     response_content = call_azure_openai(system_prompt, user_message)
    
#     if response_content is None:
#         error_msg = "No se pudo obtener respuesta de Azure OpenAI"
#         print(f"⚠️ {error_msg}")
#         return json.dumps({"en": error_msg, "es": error_msg})
    
#     try:
#         # Limpiar la respuesta para asegurar que es un JSON válido
#         cleaned_response = response_content.strip().replace("```json", "").replace("```", "")
        
#         # Valida y carga el JSON
#         parsed_json = json.loads(cleaned_response)
#         return json.dumps(parsed_json, ensure_ascii=False)

#     except Exception as e:
#         print(f"⚠️ Error procesando la respuesta de Azure OpenAI: {e}")
#         return json.dumps({"en": f"Error processing: {e}", "es": f"Error procesando: {e}"})

# def process_with_ai(html_content, prompt_template):
#     """Procesa el contenido HTML con Azure OpenAI para obtener un JSON."""
#     if not html_content or pd.isna(html_content):
#         return json.dumps({"en": "", "es": ""})
    
#     cleaned_html = clean_html(html_content)
#     orig_size = len(html_content)
#     new_size = len(cleaned_html)
#     print(f"  Reducción HTML: {orig_size} → {new_size} caracteres (-{100*(1-new_size/orig_size):.1f}%)")
    
#     system_prompt = prompt_template.split("---")[0].strip()
#     user_message = f"CONTENIDO HTML A PROCESAR:\n{cleaned_html}"
    
#     response_content = call_azure_openai(system_prompt, user_message)
    
#     if response_content is None:
#         error_msg = "No se pudo obtener respuesta de Azure OpenAI"
#         print(f"⚠️ {error_msg}")
#         return json.dumps({"en": error_msg, "es": error_msg})
    
#     try:
#         # Mejor manejo de posibles bloques de código en la respuesta
#         cleaned_response = response_content.strip()
        
#         # Intentar extraer JSON si está dentro de un bloque de código
#         if cleaned_response.startswith("```json") and cleaned_response.endswith("```"):
#             cleaned_response = cleaned_response[7:-3].strip()
#         elif cleaned_response.startswith("```") and cleaned_response.endswith("```"):
#             cleaned_response = cleaned_response[3:-3].strip()
        
#         # Validar si es JSON válido
#         parsed_json = json.loads(cleaned_response)
#         return json.dumps(parsed_json, ensure_ascii=False)

#     except Exception as e:
#         # Guardar respuesta problemática para depuración
#         error_id = f"error_{int(time.time())}"
#         with open(f"{error_id}.txt", "w", encoding="utf-8") as f:
#             f.write(f"System Prompt: {system_prompt}\n\n")
#             f.write(f"User Message: {user_message[:1000]}...\n\n")
#             f.write(f"Response Content:\n{response_content}")
        
#         print(f"⚠️ Error procesando respuesta (ID: {error_id}): {e}")
#         return json.dumps({
#             "en": f"Error processing content (ref: {error_id})",
#             "es": f"Error procesando contenido (ref: {error_id})"
#         }, ensure_ascii=False)

def process_with_ai(content_dict, prompt_template):
    """Procesa el diccionario de contenidos con Azure OpenAI para obtener un JSON anidado."""
    # Convertimos el diccionario a un string JSON para enviarlo
    user_message = json.dumps(content_dict, ensure_ascii=False)
    
    system_prompt = prompt_template.split("---")[0].strip()
    
    response_content = call_azure_openai(system_prompt, user_message)
    
    if response_content is None:
        error_msg = "No se pudo obtener respuesta de Azure OpenAI"
        # Devolvemos la estructura vacía esperada para no romper el flujo
        return {"summary_data": {}, "editor_content": {}}
    
    try:
        cleaned_response = response_content.strip()
        if cleaned_response.startswith("```json"):
            cleaned_response = cleaned_response[7:-3].strip()
        
        parsed_json = json.loads(cleaned_response)
        return parsed_json # Devuelve el diccionario Python parseado

    except Exception as e:
        print(f"⚠️ Error procesando la respuesta de Azure OpenAI: {e}")
        # Devolvemos la estructura vacía esperada
        return {"summary_data": {}, "editor_content": {}}

# def transform_data(df):
#     """Transforma el DataFrame extraído al formato de la nueva tabla 'projects'."""
#     print("\nIniciando Fase 2: Transformación de datos...")
    
#     if df.empty:
#         print("No hay datos para transformar.")
#         return []
#     # Limit
#     # df = df.iloc[0:1]

#     prompt_template = load_prompt_template()
    
#     transformed_records = []
    
#     for index, row in df.iterrows():
#         print(f"  Procesando registro {index + 1}/{len(df)} (ID antiguo: {row['id']})...")
        
#         text_editor_content_json = process_with_ai(row['body'], prompt_template)
        
#         # Mapea los campos antiguos a la nueva estructura JSON de Laravel
#         record = {
#             "slug": json.dumps({"en": slugify(row.get('title', '')), "es": slugify(row.get('title', ''))}, ensure_ascii=False), # Asumiendo el mismo slug
#             "title": json.dumps({"en": row.get('title', ''), "es": row.get('title', '')}, ensure_ascii=False), # Placeholder, puedes querer traducir el es
#             "excerpt": json.dumps({"en": row.get('excerpt', ''), "es": row.get('summary_es', '')}, ensure_ascii=False),
#             "donation_iframe": json.dumps({"en": row.get('IFRAME', ''), "es": row.get('IFRAMEES', '')}, ensure_ascii=False),
#             "video_iframe": json.dumps({"en": row.get('video', ''), "es": row.get('videoes', '')}, ensure_ascii=False),
#             "meta": json.dumps({"en": row.get('meta_description', ''), "es": ""}, ensure_ascii=False), # Placeholder para meta en español
#             "content": STATIC_CONTENT_JSON,
#             "text_editor_content": text_editor_content_json,
#             "status": "draft", # Valor por defecto según la migración
#             "user_id": 1, # ¡IMPORTANTE! Asigna el ID de usuario correcto
#             "social_links": json.dumps({}), # Placeholder
#             # timestamps (created_at, updated_at) son manejados por Laravel/MySQL
#         }
#         transformed_records.append(record)
        
#     print("✅ Transformación completada.")
#     return transformed_records

# databasemigrationprojects.ipynb

# def transform_data(df):
#     """Transforma el DataFrame extraído al formato de la nueva tabla 'projects'."""
#     print("\nIniciando Fase 2: Transformación de datos...")
#     if df.empty:
#         return []
    
#     df = df.iloc[0:2]

#     prompt_template = load_prompt_template()
#     transformed_records = []
    
#     for index, row in df.iterrows():
#         print(f"  Procesando registro {index + 1}/{len(df)} (ID antiguo: {row['id']})...")
        
#         # 1. Unificar el contenido de entrada para la IA
#         input_content_for_ai = {
#             "summary": row.get('summary', ''),
#             "body": row.get('body', ''),
#             "problem": row.get('problem', ''),
#             "solution": row.get('solution', ''),
#             "longterm": row.get('longterm', '')
#         }
        
#         # 2. Llamar a la IA para obtener la estructura anidada
#         ai_response = process_with_ai(input_content_for_ai, prompt_template)
        
#         # 3. Extraer las partes de la respuesta de la IA de forma segura
#         summary_data = ai_response.get('summary_data', {})
#         editor_content = ai_response.get('editor_content', {})

#         # 4. Preparar la estructura 'content' final
#         # Hacemos una copia profunda para no modificar la plantilla original
#         final_content_structure = json.loads(json.dumps(STATIC_CONTENT_STRUCTURE))

#         # Inyectar el summary en inglés, con valores por defecto
#         summary_en = summary_data.get('en', {})
#         final_content_structure['en']['summary_title'] = summary_en.get('title') or "Summary"
#         final_content_structure['en']['summary_text'] = summary_en.get('text', '')

#         # Inyectar el summary en español, con valores por defecto
#         summary_es = summary_data.get('es', {})
#         final_content_structure['es']['summary_title'] = summary_es.get('title') or "Resumen"
#         final_content_structure['es']['summary_text'] = summary_es.get('text', '')

#         # 5. Mapear todos los campos para el registro final de la BD
#         record = {
#             "slug": json.dumps({"en": slugify(row.get('title', '')), "es": slugify(row.get('title', ''))}, ensure_ascii=False),
#             "title": json.dumps({"en": row.get('title', ''), "es": row.get('title', '')}, ensure_ascii=False),
#             "excerpt": json.dumps({"en": row.get('excerpt', ''), "es": row.get('summary_es', '')}, ensure_ascii=False),
#             "donation_iframe": json.dumps({"en": row.get('IFRAME', ''), "es": row.get('IFRAMEES', '')}, ensure_ascii=False),
#             "video_iframe": json.dumps({"en": row.get('video', ''), "es": row.get('videoes', '')}, ensure_ascii=False),
#             "meta": json.dumps({"en": row.get('meta_description', ''), "es": ""}, ensure_ascii=False),
            
#             # Columnas con la nueva lógica
#             "content": json.dumps(final_content_structure, ensure_ascii=False),
#             "text_editor_content": json.dumps(editor_content, ensure_ascii=False),
            
#             "status": "draft",
#             "user_id": 1,
#             "social_links": json.dumps({}),
#         }
#         transformed_records.append(record)
        
#     print("✅ Transformación completada.")
#     return transformed_records

# --- FASE 2: TRANSFORMACIÓN (MODIFICADA) ---

def transform_data(df):
    """Transforma el DataFrame usando las traducciones existentes y el nuevo prompt."""
    print("\nIniciando Fase 2: Transformación de datos...")
    if df.empty:
        return []
    
    # Puedes quitar o modificar este límite para procesar todos los registros
   

    prompt_template = load_prompt_template()
    transformed_records = []
    
    for index, row in df.iterrows():
        print(f"  Procesando registro {index + 1}/{len(df)} (ID antiguo: {row['id']})...")
        
        # MODIFICADO: Ahora 'translations_es' es un diccionario con las traducciones
        translations = row.get('translations_es', {})

        # 1. Unificar contenido para la IA, ahora con ambos idiomas
        input_content_for_ai = {
            "summary": {
                "en": row.get('summary', ''),
                # Usamos la traducción si existe, si no, el original como fallback
                "es": translations.get('summary', row.get('summary', '')) 
            },
            "body": {
                "en": row.get('body', ''),
                "es": translations.get('body', row.get('body', ''))
            },
            "problem": {
                "en": row.get('problem', ''),
                "es": translations.get('problem', row.get('problem', ''))
            },
            "solution": {
                "en": row.get('solution', ''),
                "es": translations.get('solution', row.get('solution', ''))
            },
            "longterm": {
                "en": row.get('longterm', ''),
                "es": translations.get('longterm', row.get('longterm', ''))
            }
        }
        
        # 2. Llamar a la IA (la función no cambia, solo el input que le damos)
        ai_response = process_with_ai(input_content_for_ai, prompt_template)
        
        # 3. Extraer partes de la respuesta (la función no cambia)
        summary_data = ai_response.get('summary_data', {})
        editor_content = ai_response.get('editor_content', {})

        # 4. Preparar la estructura 'content' final (la lógica de inyección no cambia)
        final_content_structure = json.loads(json.dumps(STATIC_CONTENT_STRUCTURE))
        summary_en = summary_data.get('en', {})
        final_content_structure['en']['summary_title'] = summary_en.get('title') or "Summary"
        final_content_structure['en']['summary_text'] = summary_en.get('text', '')
        summary_es = summary_data.get('es', {})
        final_content_structure['es']['summary_title'] = summary_es.get('title') or "Resumen"
        final_content_structure['es']['summary_text'] = summary_es.get('text', '')

        # 5. MODIFICADO: Mapear campos usando las traducciones existentes
        record = {
            # Usamos el slug original y el traducido si existe
            "slug": json.dumps({
                "en": slugify(row.get('title', '')), 
                "es": slugify(translations.get('slug', row.get('title', '')))
            }, ensure_ascii=False),
            "title": json.dumps({
                "en": row.get('title', ''), 
                "es": translations.get('title', row.get('title', ''))
            }, ensure_ascii=False),
            "excerpt": json.dumps({
                "en": row.get('excerpt', ''), 
                "es": translations.get('excerpt', row.get('summary_es', ''))
            }, ensure_ascii=False),
            "donation_iframe": json.dumps({
                "en": row.get('IFRAME', ''), 
                "es": row.get('IFRAMEES', '')
            }, ensure_ascii=False),
            "video_iframe": json.dumps({
                "en": row.get('video', ''), 
                "es": row.get('videoes', '')
            }, ensure_ascii=False),
            "meta": json.dumps({
                "en": row.get('meta_description', ''), 
                "es": translations.get('meta_description', '')
            }, ensure_ascii=False),
            
            "content": json.dumps(final_content_structure, ensure_ascii=False),
            "text_editor_content": json.dumps(editor_content, ensure_ascii=False),
            
            "status": "draft",
            "user_id": 1,
            "social_links": json.dumps({}),
        }
        transformed_records.append(record)
        
    print("✅ Transformación completada.")
    return transformed_records


# --- FASE 3: CARGA ---

def load_into_new_db(records):
    """Carga los registros transformados en la nueva base de datos."""
    print("\nIniciando Fase 3: Carga de datos...")
    
    if not records:
        print("No hay registros para cargar.")
        return

    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME)
    
    # La migración define las columnas de la tabla 'projects'
    insert_query = text("""
    INSERT INTO projects (
        slug, title, excerpt, donation_iframe, video_iframe, content, text_editor_content, meta, 
        status, user_id, social_links, created_at, updated_at
    )
    SELECT :slug, :title, :excerpt, :donation_iframe, :video_iframe, :content, :text_editor_content, :meta, 
           :status, :user_id, :social_links, NOW(), NOW()
    WHERE NOT EXISTS (
        SELECT 1 FROM projects WHERE slug = :slug
    )
""")
    
    try:
        with engine_new.connect() as connection:
            for record in records:
                connection.execute(insert_query, record)
            connection.commit() # Confirma la transacción
        print(f"✅ Carga completada. Se han insertado {len(records)} registros en la tabla 'projects'.")
    except Exception as e:
        print(f"❌ Error al cargar datos: {e}")

In [35]:
# print("Prueba")
    
#     # Paso 1: Extraer
# data_to_migrate = extract_from_old_db()

# test_row = data_to_migrate.iloc[0:3]
# response_content = call_azure_openai(
#     system_prompt, 
#     f"CONTENIDO HTML A PROCESAR:\n{clean_html(test_row['body'])}"
# )
# print("Respuesta cruda de Azure OpenAI:")
# print(response_content)

In [36]:

print("🚀 Iniciando script de migración de 'pages' a 'projects'.")
    
    # Paso 1: Extraer
data_to_migrate = extract_from_old_db()


# data_to_migrate.head(12)
    # Paso 2: Transformar
transformed_data = transform_data(data_to_migrate)
    
    # Paso 3: Cargar
load_into_new_db(transformed_data)
    
print("\nProceso finalizado")

🚀 Iniciando script de migración de 'pages' a 'projects'.
Iniciando Fase 1: Extracción de datos y traducciones...
✅ Extracción de 'pages' completada. Se encontraron 10 registros.
✅ Traducciones asociadas correctamente.

Iniciando Fase 2: Transformación de datos...
  Procesando registro 1/10 (ID antiguo: 7)...
⚠️ Error procesando la respuesta de Azure OpenAI: Invalid control character at: line 13 column 2867 (char 4386)
  Procesando registro 2/10 (ID antiguo: 12)...
🔁 Esperando 47 segundos por límite de tasa (429)...
  Procesando registro 3/10 (ID antiguo: 13)...
  Procesando registro 4/10 (ID antiguo: 14)...
  Procesando registro 5/10 (ID antiguo: 15)...
🔁 Esperando 29 segundos por límite de tasa (429)...
  Procesando registro 6/10 (ID antiguo: 16)...
  Procesando registro 7/10 (ID antiguo: 17)...
🔁 Esperando 35 segundos por límite de tasa (429)...
  Procesando registro 8/10 (ID antiguo: 20)...
⚠️ Error procesando la respuesta de Azure OpenAI: Invalid control character at: line 14 colum